In [81]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import csv, random, time

import contractions
from bs4 import BeautifulSoup
import unicodedata
import re
import torch.nn as nn
import torch



In [108]:
trainset = []
with open('data/snips_processed/snips.csv', 'r') as f:
    reader = csv.reader(x.replace('\0', '') for x in f)
    for line in reader:
        trainset.append(line)
trainset = np.array(trainset)

testset = trainset[10000 :]

trainset = trainset[:10000]

train_sent_en = trainset[:,1]
#train_sent_sv = trainset[:,2]
train_lab = trainset[:,0]

test_sent_en = testset[:,1]
#test_sent_sv = testset[:,2]
test_lab = testset[:,0]

labels = list(set(test_lab))
lab2id = {}
id2lab = {}

for i in range(len(labels)):
    lab2id[labels[i]] = i
    id2lab[i] = labels[i]

In [109]:
len(trainset), len(testset), trainset[4]

(10000, 3784, array(['RateBook',
        'I give The Lady Decides a rating value of 4 and a best rating of 6',
        'Jag ger Damen Bestämmer sig för ett betyg värde av 4 och en bäst betyg av 6'],
       dtype='<U187'))

In [110]:
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text


def expand_contractions(text):
    return contractions.fix(text)



def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text

@np.vectorize
def pre_process_documents(document):
    
    # lower case
    document = document.lower()
    
    # remove extra newlines (often might be present in really noisy text)
    document = document.translate(document.maketrans("\n\t\r", "   "))
    
    # remove accented characters
    document = remove_accented_chars(document)
    
    # expand contractions    
    document = expand_contractions(document)
               
    # remove special characters and\or digits    
    # insert spaces between special characters to isolate them    
    special_char_pattern = re.compile(r'([{.(-)!}])')
    document = special_char_pattern.sub(" \\1 ", document)
    document = remove_special_characters(document, remove_digits=True)  
        
    # remove extra whitespace
    document = re.sub(' +', ' ', document)
    document = document.strip()
    
    return document

In [111]:
import requests
def get_vecs(sents):
    
    r = requests.post(
            'http://localhost:8501/v1/models/universal_encoder:predict', 
            json={'inputs': {'text': sents}})

    responce = r.json() 
    return responce['outputs']

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


In [112]:
def prepare_labs(labs):
    out = []
    for lab in labs:
        out.append(lab2id[lab])
    return out

In [113]:
class Baseline(nn.Module):
    def __init__(self, in_size = 512, out_size = 7):
        super(Baseline, self).__init__()

        self.W = nn.Linear(in_size, 7)
        self.out = nn.LogSoftmax(2)
        
    def forward(self, x):
        x = self.W(x)
        return self.out(x)
    
def train(model, criterion, optimizer, labels, vectors):
    model.zero_grad()
    loss = 0
    
    vectors = torch.tensor(vectors).float()
    labels = torch.tensor(labels)
    
    model_out = model.forward(vectors)
    loss += criterion(model_out[:,0], labels)
    
    loss.backward()
    optimizer.step()
    
    return loss.item()/len(labels)

def eval(model, labels, vectors):
    with torch.no_grad():
        vectors = torch.tensor(vectors).float()
        labels = torch.tensor(labels)
    
        model_out = model.forward(vectors)
        right = 0
        
        for i  in range(len(model_out)):
            k, v = model_out[i].topk(1)
            predicted, true = v.item(), labels[i].item()
            if predicted == true:
                right +=1

                
        loss = criterion(model_out[:,0], labels)
        return loss.item(), right/len(model_out)
    

In [114]:
train_labs = prepare_labs(train_lab)
test_labs = prepare_labs(test_lab)

train_sent_en = list(pre_process_documents(train_sent_en))

train_vecs = []
for i in range(0,len(train_sent_en),100):
    train_vecs.extend(get_vecs(train_sent_en[i:i+100]))

    
test_sent_en = list(pre_process_documents(test_sent_en))

test_vecs = []
for i in range(0,len(test_sent_en),100):
    test_vecs.extend(get_vecs(test_sent_en[i:i+100]))
    
test_vecs = list(map(lambda x: np.array([x]) , test_vecs))
train_vecs = list(map(lambda x: np.array([x]) , train_vecs))

In [117]:
net = Baseline()
optimizer = torch.optim.Adam(net.parameters())
criterion = torch.nn.NLLLoss()

t = time.time()
for i in range(4001):
    loss = train(net, criterion, optimizer, train_labs, train_vecs)
    if not i% 100:
        eval_loss, acc = eval(net, test_labs, test_vecs)
        print('#{:3d}, {:5d} sec. train loss: {:.7f}, eval loss: {:.4f}, acc = {:.3f}'.format(i, int(time.time() - t), loss, eval_loss, acc))


#  0,     0 sec. train loss: 0.0001944, eval loss: 1.9351, acc = 0.250
#100,    65 sec. train loss: 0.0001263, eval loss: 1.2580, acc = 0.914
#200,   136 sec. train loss: 0.0000859, eval loss: 0.8588, acc = 0.923
#300,   200 sec. train loss: 0.0000635, eval loss: 0.6375, acc = 0.928
#400,   265 sec. train loss: 0.0000503, eval loss: 0.5077, acc = 0.934
#500,   332 sec. train loss: 0.0000419, eval loss: 0.4250, acc = 0.939
#600,   393 sec. train loss: 0.0000361, eval loss: 0.3683, acc = 0.942
#700,   468 sec. train loss: 0.0000319, eval loss: 0.3274, acc = 0.946
#800,   543 sec. train loss: 0.0000288, eval loss: 0.2964, acc = 0.950
#900,   609 sec. train loss: 0.0000263, eval loss: 0.2721, acc = 0.952
#1000,   666 sec. train loss: 0.0000243, eval loss: 0.2526, acc = 0.953
#1100,   722 sec. train loss: 0.0000226, eval loss: 0.2366, acc = 0.954
#1200,   778 sec. train loss: 0.0000213, eval loss: 0.2232, acc = 0.956
#1300,   834 sec. train loss: 0.0000201, eval loss: 0.2118, acc = 0.957
#1

KeyboardInterrupt: 

In [98]:
train_vecs[0].shape

(1, 512)

In [99]:
nptv2s = np.array(train_vecs)

In [116]:
np.save('data/snips_processed/USE-en-train',np.array(train_vecs))

In [120]:
np.save('data/snips_processed/train-labs',np.array(train_labs))

In [119]:
torch.save(net, 'models/USE-linear')
